In [ ]:
import dash                     # pip install dash
from dash.dependencies import Input, Output, State
from dash import dcc
import plotly

import numpy as np

import plotly.graph_objs as go
import plotly.express as px

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import cufflinks as cf

#import dash_html_components as html
from dash import html
import plotly.express as px     # pip install plotly==5.2.2

import pandas as pd

In [ ]:
df1 = pd.read_csv('Group2Data.csv')

In [ ]:
df1.rename(columns={'energy_cons': 'Energy Consumption','employees': 'Total Employee','waste': 'Waste Produced',
                    'waste_recycled': 'Waste Recycled',
                   'water_cons': 'Water Consumption','waste_water':'Water Wastage','renewable_energy_pct':
                   'Renewable Energy','fuel_fleet':'Fuel Fleet','contrib_political':'Political Contribution',
                   'waste_recycled_pct':'Waste Recycled Percentage',' legal_spending':
                   'Legal Spending','fines_spending':'Fine Spending','employee_turnover':'Employee Turnover',
                   'female_pct' :'Female Percentage','female_mgmt_pct':'Female Management Percentage',
                   'employee_parental_pct':'Employee Parental Percentage','employee_tenure':'Employee Tenure',
                   'employee_under30_pct':'Employee Under30 Percentage',
                   'employee_over50_pct':'Employee Over50 Percentage',
                   'training_spending':'Training spending'}, inplace=True)
df1

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


indicators = ['Total Employee','EmployeeTurnover', 'Female Percentage', 'Female Management Percentage',
              'Employee Under30 Percentage','Employee Over50 Percentage',
             'Water Consumption','Waste Produced','scope_1','scope_2','scope_3']




app.layout = html.Div([
    html.H1("German sustainablity model", style={"textAlign":"center"}),
    html.Hr(),
    html.P("By Industry"),
    html.Div(html.Div([
        dcc.Dropdown(id='indus_list', clearable=False,
                     value=["IT"],
                     options=[{'label': x, 'value': x} for x in
                              df1["Industry"].unique()],style={'width': '50%'}
                    )
    ],className="two columns"),className="row"),
    
    html.P("Indicators"),
    html.Div(html.Div([
        dcc.Dropdown(id='indi_list', clearable=False,
                     value="employees",
                     options=[{'label': x, 'value': x} for x in
                              indicators],style={'width': '50%'}),
    ],className="two columns"),className="row"),
    
    html.Div(id="output-div", children=[]),
    

    html.H2("Averages per Industry", style={"textAlign":"center"}),
    html.P("Select Industry"),
    html.Div(html.Div([
        dcc.Dropdown(id='indus_list2', clearable=False,
                     value="Industrials",
                     options=[{'label': x, 'value': x} for x in
                              df1["Industry"].unique()],multi=True,style={'width': '50%'}),
    ],className="one columns"),className="row"),
    html.P("Select Indicator"),
    html.Div(html.Div([
        dcc.Dropdown(id='indi_list2', clearable=False,
                     value="employees",
                     options=[{'label': x, 'value': x} for x in
                              indicators],style={'width': '50%'}),
    ],className="one columns"),className="row"),
    html.Div(id="output-div2", children=[])
    
    
])

In [5]:
@app.callback(Output(component_id="output-div", component_property="children"),
              
              Input(component_id="indus_list", component_property="value"),
              Input(component_id="indi_list", component_property="value")
              
               
)



def make_graphs(indus_list,ind_chosen):
    
    
    
    #graph1
    df_ind = df1[df1["Industry"]==indus_list]
    
    # LINE CHART
    df_line = df_ind.sort_values(by=[ind_chosen], ascending=True)
    df_line = df_line.groupby(
        ["year","name", ind_chosen]).size().reset_index(name="count")
    #print(df_line.head())
    fig = px.line(df_line, x="year", y=ind_chosen,color='name',markers=True)
    
    return [
        
         
        html.Div([
           html.Div([dcc.Graph(figure=fig)],style={'width': '70%'}, className="six columns"),
        ], className="row"),
        
       
        
    ]
    
    
    #graph2
@app.callback(Output(component_id="output-div2", component_property="children"),            
        
              Input(component_id="indus_list2", component_property="value"),
              Input(component_id="indi_list2", component_property="value")
)

def avg_graph(indus_list2,ind_chosen2):
    
    #meandf = df.groupby(['Industry', 'years']).mean()
    
    # LINE CHART
    #df_line2 = df.sort_values(by=[ind_chosen2], ascending=True)
    #df_line2 = df_line2.groupby(
     #   ["Industry","years", ind_chosen2]).mean()
    #print(df_line.head())
    #fig2 = px.line(df_line2, x="donations", y="years",color='Industry',markers=True)
    
    df_ind1 = df1[df1["Industry"].isin(indus_list2)]
    
    # LINE CHART
    meandf= df_ind1.groupby(['Industry', 'year']).mean().reset_index()
    #print(df_line.head())
    fig2 = px.line(meandf, x='year', y=ind_chosen2,color='Industry',markers=True)
    
    #fig2_1 = px.line(meandf, x='Industry', y=ind_chosen2,color='Industry',markers=True)
    meandf1= df1.groupby(['Industry']).mean().reset_index()
    fig2_1 = px.bar(meandf1, x="Industry", y=ind_chosen2, 
                 color="Industry" )
    return [
        
         
        html.Div([
           html.Div([dcc.Graph(figure=fig2)],style={'width': '90%'}, className="four columns"),
        ], className="row"),
        html.Div([
           html.Div([dcc.Graph(figure=fig2_1)], className="four columns"),
        ], className="row"),
        
        
       
        
    ]
    
    


if __name__ == '__main__':
    app.run_server(debug=False)
